### Research Ideas:
- Predict (collector) vinyl price
- https://blog.discogs.com/en/discogs-top-100-most-expensive-records/
- limits: only 100 data points
- can we obtain # copies/presses

In [116]:
url = 'http://web.archive.org/web/20180502225137/https://blog.discogs.com/en/discogs-top-100-most-expensive-records/'

response = requests.get(url)

In [117]:
response.status_code

200

In [123]:
url='https://blog.discogs.com/en/top-100-expensive-us/'
response=requests.get(url)
response.status_code

403

In [121]:
url = 'https://www.popsike.com/php/quicksearch.php?top25=all'
response = requests.get(url)
response.status_code

200

#### 1. setup

In [5]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# with open('test_webpage/page.html') as page:
#     test_html = page.read()
# soup = BeautifulSoup(test_html, 'lxml')

#### obtain url

In [7]:
#url = 'https://www.boxofficemojo.com/title/tt0118715/' 
url = 'https://www.boxofficemojo.com/title/tt0086879/?ref_=bo_se_r_1'

In [8]:
response = requests.get(url)

In [11]:
response.status_code  #200 = success!

200

In [13]:
response.text[:1000]  #First 1000 characters of the HTML

'<!doctype html><html class="a-no-js" data-19ax5a9jf="dingo"><head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0" />\n                <meta charset="utf-8" />\n            <title dir="ltr">Amadeus - Box Office Mojo</title><meta content="Amadeus" name="title" />\n            <meta content="Box Office Mojo" property="og:site_name" />\n            <meta name="format-detection" content="telephone=no" />\n            <link href="https://m.media-amazon.com/images/G/01/boxofficemojo/v2/favicon._CB448965889_.ico" type="image/x-icon" rel="icon" />\n            <link rel="stylesheet" href="https://images-na.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|012LjolmrML.css,41t4Ok2rjiL.css,51IB+wfP8qL.css,01evdoiemkL.css,01oDR3IULNL.css,01Vctty9pOL.css,31zpKVx8wkL.css,01XPHJk60-L.css,21qPwhPKAAL.css,01Jb-VvL4uL.css,21LK7jaicML.css,11L58Qpo0GL.css,21kyTi1FabL.css,01ruG+gDPFL.css,01egbgkonWL.css,21GwE

#### Page (in this context) will give you FULL HTML text

In [14]:
page = response.text

In [15]:
soup = BeautifulSoup(page, "lxml")

In [ ]:
print(soup.prettify()) ## Makes the code nice

#### .find gives you FIRST instance of tag

In [23]:
soup.find('a')

<a class="a-link-normal" href="/?ref_=bo_nb_tt_mojologo"></a>

#### .find_all gives ALL instances. you can select instance

In [18]:
soup.find_all('a')[-1]

<a class="a-link-normal" href="/privacy/?ref_=bo_ft_tt_privacy">Privacy Policy</a>

#### Extracting things like HREF from an element in HTML

In [24]:
testing = soup.find_all('a')[-1]

In [25]:
testing.get("href")

'/privacy/?ref_=bo_ft_tt_privacy'

#### MPAA Rating, Release Date

_**STEP 1:** Create function to grab values_ 

The text matching method can also help us get runtime, rating, and release date, so let's make a reuable function.

In [28]:
## allows us to use reg expressions to search fields
import re
#runtime_regex = re.compile('Run')
#soup.find(text=runtime_regex)

In [39]:
def get_movie_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [40]:
# runtime
runtime = get_movie_value(soup,'Run')
print(runtime)

2 hr 40 min


In [41]:
# rating
rating = get_movie_value(soup,'MPAA')
print(rating)

R


In [42]:
release_date = get_movie_value(soup,'Release Date')
print(release_date)

September 21, 1984
            (Domestic)


In [43]:
release_date = release_date.split('\n')[0]  #Select the only the date
print(release_date)

September 21, 1984


In [45]:
genre = get_movie_value(soup,'Genres')
print(genre)

Biography
    
        Drama
    
        History
    
        Music


_**STEP 2:** Create helper functions to parse strings into appropriate data types_

The returned values all need a bit of formatting before we can work with this data.  Here are a few helper functions.

In [76]:
import dateutil.parser

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

_**STEP 3:** Apply these conversions_

Let's get these values again and format them all in one swoop. (Note: Rating is already correct as a string.)

In [75]:
raw_domestic_total_gross = dtg
domestic_total_gross = money_to_int(raw_domestic_total_gross)

raw_runtime = get_movie_value(soup,'Running')
runtime = runtime_to_minutes(raw_runtime)

raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
release_date = to_date(raw_release_date)

NameError: name 'dtg' is not defined

#### Put Results in Dictionary

Now that we have results for all five quantities, we can store them in a dictionary.

In [ ]:
headers = ['movie title', 'domestic total gross',
           'runtime (mins)', 'rating', 'release date']

movie_data = []
movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date]))

movie_data.append(movie_dict)
movie_data

### Scraping Tables

Let's take a look at the [top grisham movies](https://www.boxofficemojo.com/brand/bn3648190978/?ref_=bo_bns_table_26) of Box Office Mojo.

First request the HTML and parse it with Beautiful Soup.

In [92]:
url = 'https://www.boxofficemojo.com/brand/bn3648190978/?ref_=bo_bns_table_26'

response = requests.get(url)
page = response.text
soup = BeautifulSoup(page,"lxml")

Now find the main table; its the only `table` on the page.

In [91]:
table = soup.find('table')
table

<table class="a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated"><tr><th class="a-text-right mojo-field-type-rank mojo-sortable-column a-nowrap"><span title="Rank">Rank</span>
</th><th class="a-text-left mojo-field-type-release mojo-cell-wide mojo-sortable-column a-nowrap"><span title="Release">Release</span>
</th><th class="a-text-right mojo-field-type-money mojo-sort-column mojo-sortable-column mojo-estimatable a-nowrap"><a class="a-link-normal a-nowrap" href="?sortDir=asc&amp;sort=grossToDate&amp;ref_=bo_bn__resort#table" title="Lifetime Gross"><span class="a-color-state">Lifetime Gross</span><span class="a-letter-space"></span><span class="icon aok-relative"><i class="a-icon a-icon-expand" role="presentation"></i></span></a></th><th class="a-text-right mojo-field-type-positive_integer mojo-sortable-column a-nowrap"><a class="a-link-normal a-nowrap" href="?sort=maxNumTheaters&amp;ref_=bo_bn__resort#table" title="Max Theaters">Max Theaters<span

In [93]:
rows = [row for row in table.find_all('tr')]  # tr tag is for rows

Each row contains the information we want but requires more parsing.

In [94]:
rows[1]

<tr class="mojo-annotation-isEstimated"><td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">1</td><td class="a-text-left mojo-field-type-release mojo-cell-wide"><a class="a-link-normal" href="/release/rl2186511873/?ref_=bo_bn_table_1">The Firm</a></td><td class="a-text-right mojo-field-type-money mojo-sort-column mojo-estimatable">$158,348,367</td><td class="a-text-right mojo-field-type-positive_integer">2,393</td><td class="a-text-right mojo-field-type-money">$25,400,000</td><td class="a-text-right mojo-field-type-positive_integer">2,393</td><td class="a-text-left mojo-field-type-date a-nowrap"><a class="a-link-normal" href="/date/1993-07-02/?ref_=bo_bn_table_1">Jul 2, 1993</a></td><td class="a-text-left mojo-field-type-release_studios"><a class="a-link-normal" href="https://pro.imdb.com/company/co0023400/boxoffice/?view=releases&amp;ref_=mojo_bn_table_1&amp;rf=mojo_bn_table_1" rel="noopener" target="_blank">Paramount Pictures<svg class="mojo-new-window-svg"

Remember: you can chain methods together to look for information!

In [125]:
# Needed to change this to column 1 if you look above
rows[1].find_all('td')[1].find('a')['href']

'/release/rl2186511873/?ref_=bo_bn_table_1'

#### Now grab data for the first 10 movies with a loop.

In [100]:
movies = {}

for row in rows[1:11]:
    items = row.find_all('td')
    link = items[1].find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'The Firm': ['/release/rl2186511873/?ref_=bo_bn_table_1',
  '1',
  'The Firm',
  '$158,348,367',
  '2,393',
  '$25,400,000',
  '2,393',
  'Jul 2, 1993',
  'Paramount Pictures\n\n',
  'true'],
 'A Time to Kill': ['/release/rl3933505025/?ref_=bo_bn_table_2',
  '2',
  'A Time to Kill',
  '$108,766,007',
  '2,313',
  '$14,823,159',
  '2,123',
  'Jul 24, 1996',
  'Warner Bros.\n\n',
  'false'],
 'The Pelican Brief': ['/release/rl426280449/?ref_=bo_bn_table_3',
  '3',
  'The Pelican Brief',
  '$100,768,056',
  '2,022',
  '$16,864,404',
  '1,993',
  'Dec 17, 1993',
  'Warner Bros.\n\n',
  'false'],
 'The Client': ['/release/rl575309313/?ref_=bo_bn_table_4',
  '4',
  'The Client',
  '$92,115,211',
  '2,365',
  '$17,174,262',
  '2,052',
  'Jul 22, 1994',
  'Warner Bros.\n\n',
  'false'],
 'Christmas with the Kranks': ['/release/rl2806744577/?ref_=bo_bn_table_5',
  '5',
  'Christmas with the Kranks',
  '$73,780,539',
  '3,416',
  '$21,570,867',
  '3,393',
  'Nov 24, 2004',
  'Revolution Studios

### Scraping Multiple Pages

Now that we have the links for several Holiday movies we can visit each link to extract even more information about each movie.  Let's use `pandas` to help.

In [61]:
import pandas as pd

In [102]:
g_movies = pd.DataFrame(movies).T  #transpose
g_movies.columns
g_movies.columns = ['link_stub','rank','release','ltgross','maxtheatre','opening','openth','releasedate','distributor','estimated']

In [103]:
g_movies.head()

,link_stub,rank,release,ltgross,maxtheatre,opening,openth,releasedate,distributor,estimated
The Firm,/release/rl2186511873/?ref_=bo_bn_table_1,1,The Firm,"$158,348,367","2,393","$25,400,000","2,393","Jul 2, 1993",Paramount Pictures\n\n,true
A Time to Kill,/release/rl3933505025/?ref_=bo_bn_table_2,2,A Time to Kill,"$108,766,007","2,313","$14,823,159","2,123","Jul 24, 1996",Warner Bros.\n\n,false
The Pelican Brief,/release/rl426280449/?ref_=bo_bn_table_3,3,The Pelican Brief,"$100,768,056","2,022","$16,864,404","1,993","Dec 17, 1993",Warner Bros.\n\n,false
The Client,/release/rl575309313/?ref_=bo_bn_table_4,4,The Client,"$92,115,211","2,365","$17,174,262","2,052","Jul 22, 1994",Warner Bros.\n\n,false
Christmas with the Kranks,/release/rl2806744577/?ref_=bo_bn_table_5,5,Christmas with the Kranks,"$73,780,539","3,416","$21,570,867","3,393","Nov 24, 2004",Revolution Studios\n\n,false


We'll also combine all previous steps into one helper function.

In [108]:
def get_movie_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")

    
    headers = ['movie_title' ,# 'domestic_total_gross',
               'runtime_minutes', 'rating',] # 'release_date']
    
    #Get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()

    #Get domestic gross
    raw_domestic_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                    .find_all('span', class_='money')[0]
                                    .text
                               )
    domestic_total_gross = money_to_int(raw_domestic_total_gross)

    #Get runtime
    raw_runtime = get_movie_value(soup,'Running')
    runtime = runtime_to_minutes(raw_runtime)
    
    #Get rating
    rating = get_movie_value(soup,'MPAA')

    #Get release date
    raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
    release_date = to_date(raw_release_date)
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date
                                   ]))

    return movie_dict

In [109]:
# LINK_STUB is the added parts to the URL
g_movies_page_info_list = []

for link in g_movies.link_stub:
    g_movies_page_info_list.append(get_movie_dict(link))

In [110]:
g_movies_page_info_list

[{'movie_title': 'The Firm', 'runtime_minutes': 158348367, 'rating': 154},
 {'movie_title': 'A Time to Kill',
  'runtime_minutes': 108766007,
  'rating': 149},
 {'movie_title': 'The Pelican Brief',
  'runtime_minutes': 100768056,
  'rating': 141},
 {'movie_title': 'The Client', 'runtime_minutes': 92115211, 'rating': 119},
 {'movie_title': 'Christmas with the Kranks',
  'runtime_minutes': 73780539,
  'rating': 99},
 {'movie_title': 'Runaway Jury', 'runtime_minutes': 49443628, 'rating': 127},
 {'movie_title': 'The Rainmaker', 'runtime_minutes': 45916769, 'rating': 135},
 {'movie_title': 'The Chamber', 'runtime_minutes': 14551359, 'rating': 113},
 {'movie_title': 'The Gingerbread Man',
  'runtime_minutes': 1677131,
  'rating': 114},
 {'movie_title': 'Mickey', 'runtime_minutes': 294758, 'rating': 105}]

In [111]:
g_movies_page_info = pd.DataFrame(g_movies_page_info_list)  #convert list of dict to df
g_movies_page_info.set_index('movie_title', inplace=True)

g_movies_page_info

,runtime_minutes,rating
movie_title,,
The Firm,158348367,154
A Time to Kill,108766007,149
The Pelican Brief,100768056,141
The Client,92115211,119
Christmas with the Kranks,73780539,99
Runaway Jury,49443628,127
The Rainmaker,45916769,135
The Chamber,14551359,113
The Gingerbread Man,1677131,114


(Note: the rating is indeed missing from a few of these pages!  How could you fix that?)

We can now match this back up with the movie information collected from the table by merging these dataframes.

In [77]:
g_movies = g_movies.merge(g_movies_page_info, left_index=True, right_index=True)

g_movies

,link_stub,title,rank_g_movies,lifetime_gross,rank_overall,year,domestic_total_gross,runtime_minutes,rating,release_date
Toy Story 4,/title/tt1979376/?ref_=bo_cso_table_1,Toy Story 4,1,"$434,038,008",24,2019,434038008,100,G,2019-06-20
The Lion King,/title/tt0110357/?ref_=bo_cso_table_2,The Lion King,2,"$422,783,777",28,1994,422783777,88,G,1994-06-15
Toy Story 3,/title/tt0435761/?ref_=bo_cso_table_3,Toy Story 3,3,"$415,004,880",30,2010,415004880,103,None,2010-06-16
Finding Nemo,/title/tt0266543/?ref_=bo_cso_table_4,Finding Nemo,4,"$380,843,261",43,2003,380843261,100,None,2003-05-30
"Monsters, Inc.",/title/tt0198781/?ref_=bo_cso_table_5,"Monsters, Inc.",5,"$290,642,256",100,2001,290642256,92,G,2001-11-02
